# Cobwebbing

Adam Rumpf

Created 4/17/21

Based on a <a href="https://github.com/adam-rumpf/mathematica-class-demonstrations#cobwebbing" target="_blank">Mathematica class demonstration</a>.

This is a standalone widget for playing around with cobweb diagrams for various dynamical systems. See the full notebook [here](./cobwebbing.ipynb).

See more Jupyter Notebook class demonstrations <a href="https://github.com/adam-rumpf/jupyter-class-demonstrations" target="_blank">here</a>.

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define parameters
COBWEB_MAX = 10 # number of cobweb iterations to generate

# Define functions

def lmap_abs(x, lim=1.0, r=1.0, h=0.0):
    """Discrete logistic map with absolute harvesting.
    
    Positional arguments:
    x - input value
    
    Keyword arguments:
    lim (1.0) - population limit
    r (1.0) - intrinsic growth rate
    h (0.0) - harvesting rate
    """
    
    return r*x*(lim-x) - h

def lmap_rel(x, lim=1.0, r=1.0, h=0.0):
    """Discrete logistic map with relative harvesting.
    
    Positional arguments:
    x - input value
    
    Keyword arguments:
    lim (1.0) - population limit
    r (1.0) - intrinsic growth rate
    h (0.0) - harvesting rate
    """
    
    return max(r-h, 0.0)*x*(lim-x)

def cobweb_update(x0, lim=1.0, r=1.0, h=0.0, mode=0):
    """Updates the global cobweb lists.
    
    Positional arguments:
    x0 - initial population value
    
    Keyword arguments:
    lim (1.0) - population limit
    r (1.0) - intrinsic growth rate
    h (0.0) - harvesting rate
    mode (0) - 0 for absolute harvesting, 1 for relative harvesting
    """
    
    global cwx, cwy
    
    # Initialize cobweb coordinates
    cwx[0] = x0
    cwy[0] = 0.0
    
    # Absolute harvesting
    if mode == 0:
        for i in range(0, 2*COBWEB_MAX, 2):
            cwx[i+1] = cwx[i]
            cwy[i+1] = max(lmap_abs(cwx[i], r=r, h=h), 0.0)
            cwx[i+2] = cwy[i+1]
            cwy[i+2] = cwy[i+1]
    
    # Relative harvesting
    elif mode == 1:
        for i in range(0, 2*COBWEB_MAX, 2):
            cwx[i+1] = cwx[i]
            cwy[i+1] = max(lmap_rel(cwx[i], r=r, h=h), 0.0)
            cwx[i+2] = cwy[i+1]
            cwy[i+2] = cwy[i+1]

# Set up plot
fig, ax = plt.subplots()

# Generate x values
x = np.linspace(0, 1.5, 101)

# Initialize cobweb lists
cwx = np.zeros(2*COBWEB_MAX+1) # cobweb x-coordinates
cwy = np.zeros_like(cwx) # cobweb y-coordinates

# Draw plot lines
@widgets.interact(mode=(0, 1, 1), r=(0.5, 4.0, 0.1), h=(0.0, 1.0, 0.05), x0=(0.0, 1.25, 0.01), step=(0, COBWEB_MAX, 1))
def update1(mode=0, r=2.0, h=0.0, x0=0.25, step=np.math.floor(COBWEB_MAX/4)):
    global ax, cwx, cwy
    cobweb_update(x0, r=r, h=h, mode=mode)
    ax.clear()
    ax.set_ylim([0, 1.5])
    ax.grid(False)
    ax.set_xlabel("$x_n$")
    ax.set_ylabel("$x_{n+1}$")
    if mode == 0:
        plt.title("Discrete Logistic Growth with Absolute Harvesting")
        ax.plot(x, lmap_abs(x, r=r, h=h), color="C0")
    elif mode == 1:
        plt.title("Discrete Logistic Growth with Relative Harvesting")
        ax.plot(x, lmap_rel(x, r=r, h=h), color="C0")
    ax.plot(x, x, color="black")
    ax.plot(cwx[:2*step], cwy[:2*step], color="C1")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='mode', max=1), FloatSlider(value=2.0, description='r', m…